In [1]:
%run GeoInfo_functions.ipynb

In [2]:
import os
import datetime as dt
import calendar
from sodapy import Socrata
from functions import *

# **<font color = 'DarkRed'> County Level </font>**

## **<font color='Black'> Obtain COVID data </font>**

## **<font color='Black'> Obtain COVID data from the Virginia Health Department Website </font>**

<font color='red'>First check if the json file "VDH-COVID-19-PublicUseDataset-Cases.json" exists.<br>
If it exists and is current then read the file.<br>Otherwise, import the data from the Virginia Health Department Website and save as a json file</font>

In [3]:
def Get_VA_COVID_data(ByPass_Update = False):
    VDH_filename = 'data/VDH-COVID-19-PublicUseDataset-Cases.json'
    
    if os.path.isfile(VDH_filename):
        print('Yay! VDH file exists')
        # check if file is current
        filetime = dt.datetime.fromtimestamp(os.path.getmtime(VDH_filename))
        print("VDH file was last modified on ", filetime.date())
        today = dt.datetime.now().date()
        print("Today's date is: ", today)
        if filetime.date() == today:
            print('Yay, VDH file is current!')
            COVID_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
        else:
            print('VDH file is not current')
            if ByPass_Update:
                print('Reading existing file without updating')
                COVID_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
            else:
                COVID_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
                COVID_data_Virginia.sort_values(by = 'report_date', ignore_index = True, inplace = True)
                
                date = COVID_data_Virginia['report_date'].iloc[-1]
                print('Last recorded date on the existing VDH file is ', pd.to_datetime(date, yearfirst = True))
            
                print('Updating VDH file from the Virginia Health Department Website')
                
                df = Update_VDH_COVID_Data(date)
                mask = COVID_data_Virginia['report_date'] < date
                COVID_data_Virginia = pd.concat([COVID_data_Virginia[mask], df], ignore_index = True)
                COVID_data_Virginia.to_json(VDH_filename, orient = 'table')
    else:
        print('VDH file does not exist')
        print('Importing VDH file from the Virginia Health Department Website')
        COVID_data_Virginia = Import_VDH_COVID_Data()
        COVID_data_Virginia.to_json(VDH_filename, orient = 'table')
        
    COVID_data_Virginia['report_date'] = pd.to_datetime(COVID_data_Virginia['report_date']).dt.strftime('%y/%m/%d')
    COVID_data_Virginia['County Code'] = COVID_data_Virginia['fips'].astype(str).str[2:]
    COVID_data_Virginia['fips'] = COVID_data_Virginia['fips'].astype(int)

    cols = ['total_cases', 'hospitalizations', 'deaths']
    COVID_data_Virginia[cols] = COVID_data_Virginia[cols].apply(pd.to_numeric, errors='coerce')

    # The Thomas Jefferson health district was later renamed Blue Ridge
    COVID_data_Virginia['vdh_health_district'] = np.where(COVID_data_Virginia['vdh_health_district'] == 'Thomas Jefferson',
                                                          'Blue Ridge', COVID_data_Virginia['vdh_health_district'])

    # At one instance there is an error where "c" was entered for the vdh_health_district instead of Rappahannock Rapidan
    COVID_data_Virginia['vdh_health_district'] = np.where(COVID_data_Virginia['vdh_health_district'] == 'c', 
                                                          'Rappahannock Rapidan', COVID_data_Virginia['vdh_health_district'])
    
#     COVID_data_Virginia['locality'] = [(Get_CountyNames_Dict('51'))[x] for x in COVID_data_Virginia['County Code']]
    
    COVID_data_Virginia.columns = ['Report Date', 'fips', 'County Name', 'Health District', 
                                   'Total Cases', 'Hospitalizations', 'Deaths', 'County Code']
    
    return COVID_data_Virginia

### <font color='red'> Correct for negative data after completion of quality assurance by the state's Department of Health </font>
* <font color='royalblue'> Sometimes the total cases, hospitalization, or death decreases after the DH conducts quality assurance to ensure:<br>
1) cases are not assigned to the wrong locality as some ZIP codes cross between localities <br>
2) multiple positive test results for the same infection in one person are not counted as multiple COVID-19 cases <br>
3) the case follows the criteria outlined in a national case surveillance definition by the CDC. </font>

In [4]:
def Clean_COVID_data(df, col_list):
    VDH_filename = 'data/VDH-COVID-19-PublicUseDataset-Cases_cleaned.json'
    df.sort_values(by = ['County Code', 'Report Date'], inplace = True)
    date0 = df.iloc[0]['Report Date']
    df.set_index(['County Code', 'Report Date'], inplace = True)
    df.sort_index(inplace = True)
    
    def Get_Dates_of_Adjusment(Mask):
        try:
            Adjusted_dates = (df[Mask].index).tolist()
        except KeyError:
            Adjusted_dates = []
        return Adjusted_dates
    
    for col in col_list:
        new_col = 'New Daily ' + col.replace('Total ', '')
        df[new_col] = df.groupby('County Code')[col].diff().fillna(0)

        print('Cleaning', col, 'data')
        Adjusted_dates = Get_Dates_of_Adjusment(df[new_col] < 0)
        iter = 0
        for d in Adjusted_dates:
            Max_cases = df.loc[d, col]
            Mask = df[col] > Max_cases
            Ind = df[Mask].loc[(d[0], date0) : d].index
            df.loc[Ind, col] = Max_cases
            iter += 1
            
        print(iter, ' entries adjusted')
        df[new_col] = df.groupby('County Code')[col].diff().fillna(0).astype('int')
    
    df.reset_index(inplace = True)
    df.to_json(VDH_filename, orient = 'table')
    
    # Keep track of last full clean
    filename = 'data/FullCleaning_Date.txt'
    with open(filename, 'w') as file:
        file.write(str(dt.datetime.now().date()))
    
    return df

In [5]:
def Update_Clean_COVID_data(df_all, col_list):
    VDH_filename = 'data/VDH-COVID-19-PublicUseDataset-Cases_cleaned.json'
    
    df_clean = pd.read_json(VDH_filename, orient = 'table')
    last_date = df_clean.iloc[-1]['Report Date']
    print('Last recorded date on the existing cleaned VDH file is ', pd.to_datetime(last_date, yearfirst = True))
    
    df = (df_all[df_all['Report Date'] >= last_date]).copy()
    df0 = df_clean[df_clean['Report Date'] < last_date].copy()
    df0.drop(['New Daily Cases', 'New Daily Hospitalizations', 'New Daily Deaths'], axis = 1, inplace = True)
    
    df = pd.concat([df0, df], ignore_index = True)
    df.sort_values(by = ['County Code', 'Report Date'], inplace = True)
    date0 = df.iloc[0]['Report Date']
    df.set_index(['County Code', 'Report Date'], inplace = True)
    df.sort_index(inplace = True)
    
    def Get_Dates_of_Adjusment(Mask):
        try:
            Adjusted_dates = (df[Mask].index).tolist()
        except KeyError:
            Adjusted_dates = []
        return Adjusted_dates
    
    for col in col_list:
        new_col = 'New Daily ' + col.replace('Total ', '')
        df[new_col] = df.groupby('County Code')[col].diff().fillna(0)

        print('Cleaning', col, 'data')
        Adjusted_dates = Get_Dates_of_Adjusment(df[new_col] < 0)

        iter = 0
        for d in Adjusted_dates:
            Max_cases = df.loc[d, col]
            Mask = df[col] > Max_cases
            Ind = df[Mask].loc[(d[0], date0) : d].index
            df.loc[Ind, col] = Max_cases
            iter += 1

        print(iter, ' entries adjusted')
        df[new_col] = df.groupby('County Code')[col].diff().fillna(0).astype('int')
        
    df.reset_index(inplace = True)
    df.sort_values(by = ['County Code', 'Report Date'], inplace = True)
    df.to_json(VDH_filename, orient = 'table')
    
    return df

### <font color='red'> First Go through preliminary cleaning </font>
* <font color='royalblue'> It looks like after 01/31/2023 deaths and hospitalizations data suddenly dropped by half. <br>
Here, I assume that all data before 02/01/2023 were double counted and will divide by half. <br>
Next, I correct for sharp peaks where the total increases and then immediately decreases the next day or vice versa. </font>

In [6]:
def Prelim_Clean_COVID_Data(df_orig):
    df = df_orig.copy()
    # first correct for the presumed double counting prior to feb 01 2023
    for m in ['Hospitalizations', 'Deaths']:
        df[m] = np.where(df['Report Date'] < '23/02/01', (df[m]/2).astype(int), df[m])
    
    # next correct for sharp peaks
    df.set_index(['County Code', 'Report Date'], inplace = True)
    df.sort_index(inplace = True)
    date_last = df.iloc[-1][0]
    
    col_list = ['Total Cases', 'Hospitalizations', 'Deaths']
    for col in col_list:
        dcol = 'Diff ' + col
        dcol_sum = 'Sum Diff ' + col
        df[dcol] = df.groupby(level = 'County Code')[col].diff().fillna(0).astype(int)
        df[dcol_sum] = (df.groupby(level = 'County Code', as_index = False)[dcol].rolling(2).sum().
                        shift(periods = -1).fillna(0).astype(int).drop('County Code', axis = 1))
        mask1 = abs(df[dcol_sum]) < 0.11*abs(df[dcol])
        mask2 = df.index.get_level_values('Report Date') != date_last
        mask = mask1 & mask2
        Nentries = len(df[mask])
        if Nentries == 0:
            print('No entries to be adjusted for ', col)
            continue
        df.loc[mask, col] = np.nan
        df[col] = (df[col].ffill()).astype(int)
        print('{} entries adjusted for {}'.format(Nentries, col))
        df.drop(columns = [dcol, dcol_sum], inplace = True)
        
    df.reset_index(inplace = True)  
    return df

In [7]:
def Get_Cleaned_VA_COVID_data(ByPass_Update = False):
    VDH_filename = 'data/VDH-COVID-19-PublicUseDataset-Cases_cleaned.json'
    clean_date_filename = 'data/FullCleaning_Date.txt'

    today = dt.datetime.now().date()
    print("Today's date is: ", today)
    
    if os.path.isfile(clean_date_filename):
        with open(clean_date_filename, 'r') as file:
            clean_date_str = file.read()
        
        try:
            clean_date = dt.datetime.strptime(clean_date_str, '%Y-%m-%d').date()
            print('last full clean date: ', clean_date)
            if (today - clean_date).days < 14:
                print('Not due for full cleaning')
                full_clean = False
            else:
                print('Due for full cleaning')
                full_clean = True
        except:
            full_clean = True
    else:
        full_clean = True
    
    col_list = ['Total Cases', 'Hospitalizations', 'Deaths']
    if full_clean:
        print('time for full cleaning')
        df = Get_VA_COVID_data(ByPass_Update)
        df = Prelim_Clean_COVID_Data(df)
        COVID_data_Virginia = Clean_COVID_data(df, col_list)
    else:        
        if os.path.isfile(VDH_filename):
            print('Yay! cleaned VDH file exists')
            # check if file is current
            filetime = dt.datetime.fromtimestamp(os.path.getmtime(VDH_filename))
            print("cleaned VDH file was last modified on ", filetime.date())
    #         today = dt.datetime.now().date()
    #         print("Today's date is: ", today)
            if filetime.date() == today:
                print('Yay, cleaned VDH file is current!')
                COVID_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
            else:
                print('cleaned VDH file is not current')
                if ByPass_Update:
                    print('Reading existing file without updating')
                    COVID_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
                else:
                    print('retrieving VDH file')
                    df = Get_VA_COVID_data(ByPass_Update)
                    print('Updating Cleaned VDH file')
    #                 col_list = ['Total Cases', 'Hospitalizations', 'Deaths']
                    COVID_data_Virginia = Update_Clean_COVID_data(df, col_list)
        else:
            print('cleaned VDH file does not exist')
            df = Get_VA_COVID_data(ByPass_Update)
            # First go through the preliminary cleaning
            df = Prelim_Clean_COVID_Data(df)
            COVID_data_Virginia = Clean_COVID_data(df, col_list)
        
    COVID_data_Virginia['County Code'] = COVID_data_Virginia['County Code'].astype(str)
    
    COVID_data_Virginia['Hospitalizations Ratio'] = (COVID_data_Virginia['Hospitalizations'].
                                                     divide(COVID_data_Virginia['Total Cases'])).replace(np.nan, 0)
    COVID_data_Virginia['Deaths Ratio'] = (COVID_data_Virginia['Deaths'].
                                           divide(COVID_data_Virginia['Total Cases'])).replace(np.nan, 0)
    COVID_data_Virginia['Deaths per Hospitalizations'] = (COVID_data_Virginia['Deaths'].
                                                          divide(COVID_data_Virginia['Hospitalizations'])
                                                         ).replace(np.nan, 0)
        
    return COVID_data_Virginia

# Get Vaccine Data

In [8]:
def Get_VA_COVID_Vaccines_data(ByPass_Update = False):
    VDH_filename = 'data/VDH-COVID-19-PublicUseDataset-Vaccines-DosesAdministered.json'
    
    if os.path.isfile(VDH_filename):
        print('Yay! VDH Vaccines Data file exists')
        # check if file is current
        filetime = dt.datetime.fromtimestamp(os.path.getmtime(VDH_filename))
        print("VDH Vaccines Data file was last modified on ", filetime.date())
        today = dt.datetime.now().date()
        print("Today's date is: ", today)
        if filetime.date() == today:
            print('Yay, VDH Vaccines Data file is current!')
            Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
        else:
            print('VDH Vaccines Data file is not current')
            if ByPass_Update:
                print('Reading existing file without updating')
                Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
            else:
                Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
                Vaccines_data_Virginia.dropna(inplace = True)
                Vaccines_data_Virginia.sort_values(by = 'administration_date', ignore_index = True, inplace = True)
                
                date = Vaccines_data_Virginia['administration_date'].iloc[-1]
                print('Last recorded date on the existing VDH Vaccines Data file is ', pd.to_datetime(date))
            
                print('Updating VDH Vaccines Data file from the Virginia Health Department Website')

                df = Update_VDH_COVID_Vaccines_Data(date)
                mask = Vaccines_data_Virginia['administration_date'] < date
                Vaccines_data_Virginia = pd.concat([Vaccines_data_Virginia[mask], df], ignore_index = True)
                Vaccines_data_Virginia.to_json(VDH_filename, orient = 'table')
    else:
        print('VDH Vaccines Data file does not exist')
        print('Importing VDH Vaccines Data file from the Virginia Health Department Website')
        Vaccines_data_Virginia = Import_VDH_COVID_Vaccines_Data()
        Vaccines_data_Virginia.to_json(VDH_filename, orient = 'table')

    Vaccines_data_Virginia['administration_date'] = pd.DatetimeIndex(Vaccines_data_Virginia['administration_date'])
    Vaccines_data_Virginia['County Code'] = Vaccines_data_Virginia['fips'].astype(str).str[2:]

    cols = ['fips', 'dose_number', 'vaccine_doses_administered']
    Vaccines_data_Virginia[cols] = Vaccines_data_Virginia[cols].apply(pd.to_numeric, errors='coerce')
    Vaccines_data_Virginia.dropna(inplace = True)
    
    Vaccines_data_Virginia['fips'] = Vaccines_data_Virginia['fips'].astype(int)

    return Vaccines_data_Virginia

In [9]:
def Get_VA_COVID_Vaccines_data_By_Age(ByPass_Update = False):
    VDH_filename = 'data/VDH-COVID-19-PublicUseDataset-Vaccines-DosesAdministered_By-Age-Group.json'
    
    if os.path.isfile(VDH_filename):
        print('Yay! VDH Vaccines Data file exists')
        # check if file is current
        filetime = dt.datetime.fromtimestamp(os.path.getmtime(VDH_filename))
        print("VDH Vaccines Data file was last modified on ", filetime.date())
        today = dt.datetime.now().date()
        print("Today's date is: ", today)
        if filetime.date() == today:
            print('Yay, VDH Vaccines Data file is current!')
            Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
            print('Done Reading existing file')
        else:
            print('VDH Vaccines Data file is not current')
            if ByPass_Update:
                print('Reading existing file without updating')
                Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
            else:
                Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
                Vaccines_data_Virginia.sort_values(by = 'report_date', ignore_index = True, inplace = True)
                
                date = Vaccines_data_Virginia['report_date'].iloc[-1]
                print('Last recorded date on the existing VDH Vaccines Data file is ', pd.to_datetime(date))
            
                print('Updating VDH Vaccines Data file from the Virginia Health Department Website')
                
                df = Update_VDH_COVID_Vaccines_By_Age_Data(date)
                mask = Vaccines_data_Virginia['report_date'] < date
                Vaccines_data_Virginia = pd.concat([Vaccines_data_Virginia[mask], df], ignore_index = True)
                Vaccines_data_Virginia.to_json(VDH_filename, orient = 'table')
    else:
        print('VDH Vaccines Data file does not exist')
        print('Importing VDH Vaccines Data file from the Virginia Health Department Website')
        Vaccines_data_Virginia = Import_VDH_COVID_Vaccines_By_Age_Data()
        Vaccines_data_Virginia.dropna(inplace = True)
        Vaccines_data_Virginia.sort_values(by = 'report_date', ignore_index = True, inplace = True)
        Vaccines_data_Virginia.to_json(VDH_filename, orient = 'table')

    Vaccines_data_Virginia['report_date'] = pd.DatetimeIndex(Vaccines_data_Virginia['report_date'])
    Vaccines_data_Virginia['County Code'] = Vaccines_data_Virginia['fips'].astype(str).str[2:]
    
    Vaccines_data_Virginia.drop(columns=['health_district', 'health_region', 'age_group_type', '_18_vaccination_count'], 
                                inplace = True)
    Vaccines_data_Virginia.rename(columns={"report_date": "Report Date", "age_group": "Age Group", 
                                           "people_by_vaccination_status_count": "Count"}, 
                                  inplace = True)

    cols = ['fips', 'Count']
    Vaccines_data_Virginia[cols] = Vaccines_data_Virginia[cols].apply(pd.to_numeric, errors='coerce')
    Vaccines_data_Virginia.dropna(inplace = True)
    
    Vaccines_data_Virginia['fips'] = Vaccines_data_Virginia['fips'].astype(int)
    
    return Vaccines_data_Virginia

In [10]:
def Get_VA_COVID_Vaccines_data_By_Sex(ByPass_Update = False):
    VDH_filename = 'data/VDH-COVID-19-PublicUseDataset-Vaccines-DosesAdministered_By-Sex.json'
    
    if os.path.isfile(VDH_filename):
        print('Yay! VDH Vaccines Data file exists')
        # check if file is current
        filetime = dt.datetime.fromtimestamp(os.path.getmtime(VDH_filename))
        print("VDH Vaccines Data file was last modified on ", filetime.date())
        today = dt.datetime.now().date()
        print("Today's date is: ", today)
        if filetime.date() == today:
            print('Yay, VDH Vaccines Data file is current!')
            Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
            print('Done Reading existing file')
        else:
            print('VDH Vaccines Data file is not current')
            if ByPass_Update:
                print('Reading existing file without updating')
                Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
            else:
                Vaccines_data_Virginia = pd.read_json(VDH_filename, orient = 'table')
                Vaccines_data_Virginia.sort_values(by = 'report_date', ignore_index = True, inplace = True)
                
                date = Vaccines_data_Virginia['report_date'].iloc[-1]
                print('Last recorded date on the existing VDH Vaccines Data file is ', pd.to_datetime(date))
            
                print('Updating VDH Vaccines Data file from the Virginia Health Department Website')
                
                df = Update_VDH_COVID_Vaccines_By_Sex_Data(date)
                mask = Vaccines_data_Virginia['report_date'] < date
                Vaccines_data_Virginia = pd.concat([Vaccines_data_Virginia[mask], df], ignore_index = True)
                Vaccines_data_Virginia.to_json(VDH_filename, orient = 'table')
    else:
        print('VDH Vaccines Data file does not exist')
        print('Importing VDH Vaccines Data file from the Virginia Health Department Website')
        Vaccines_data_Virginia = Import_VDH_COVID_Vaccines_By_Sex_Data()
        Vaccines_data_Virginia.dropna(inplace = True)
        Vaccines_data_Virginia.sort_values(by = 'report_date', ignore_index = True, inplace = True)
        Vaccines_data_Virginia.to_json(VDH_filename, orient = 'table')

    Vaccines_data_Virginia['report_date'] = pd.DatetimeIndex(Vaccines_data_Virginia['report_date'])
    Vaccines_data_Virginia['County Code'] = Vaccines_data_Virginia['fips'].astype(str).str[2:]
    
    Vaccines_data_Virginia.drop(columns=['health_district', 'health_region'], inplace = True)
    Vaccines_data_Virginia.rename(columns={"report_date": "Report Date", 
                                           "people_by_vaccination_status_count": "Count", 
                                           "locality": "County Name", 
                                           "vaccination_status": "Vaccination Status"}, 
                                  inplace = True)
    
    Vaccines_data_Virginia['Vaccination Status'] = np.where(Vaccines_data_Virginia['Vaccination Status'] == 
                                                            "Booster/ Third Dose", "First Booster", 
                                                            Vaccines_data_Virginia['Vaccination Status'])

    cols = ['fips', 'Count']
    Vaccines_data_Virginia[cols] = Vaccines_data_Virginia[cols].apply(pd.to_numeric, errors='coerce')
    Vaccines_data_Virginia.dropna(inplace = True)
    
    Vaccines_data_Virginia['fips'] = Vaccines_data_Virginia['fips'].astype(int)
    
    return Vaccines_data_Virginia

In [11]:
def Get_Counties_Vacc_Summary(State_Code, df):
    df_pivot = df.pivot_table(index = ['County Code', 'administration_date'], columns = 'dose_number', 
                              values = 'vaccine_doses_administered', fill_value = 0, aggfunc = np.sum).astype(int)
    df_counties = ((df_pivot.groupby(level = 0).cumsum().add_prefix('people with at least ').add_suffix(' doses')).
                   unstack(0).fillna(method='ffill').fillna(0).astype(int)).stack().reset_index()
    df_counties = df_counties.rename(columns = {'people with at least 1 doses': 'people with at least 1 dose'})
    
    col_list = df_counties.loc[:, 'people with at least 1 dose':'people with at least 4 doses'].columns.to_list()
    df_counties = Add_Pop_data(State_Code, 'County', df_counties, col_list)
    df_counties.sort_values(by = ['administration_date', 'County Code'], inplace = True)
    df_counties.set_index('administration_date', inplace = True)
    df_counties.index.names = ['Date']
    
    df_counties['County Name'] = [(Get_CountyNames_Dict('51'))[x] for x in df_counties['County Code']]
    
    return df_counties

In [12]:
def Get_Counties_Vacc_Stat_Summary(State_Code, df):
    df_pivot = df.pivot_table(index = ['County Code', 'Report Date'], columns = 'Vaccination Status', 
                              values = 'Count', fill_value = 0, aggfunc = np.sum).astype(int)
    df0 = df_pivot.rename_axis(None, axis = 1).reset_index()
    
    df_counties = Prelim_Clean_Vacc_By_Sex_Data(df0)
    df_counties = Clean_Vacc_By_Sex_Data(df_counties)
    col_list = df_counties.set_index(['County Code', 'Report Date']).columns.to_list()
    df_counties = Add_Pop_data(State_Code, 'County', df_counties, col_list)
    df_counties.sort_values(by = ['Report Date', 'County Code'], inplace = True)
    df_counties.set_index('Report Date', inplace = True)
    df_counties.index.names = ['Date']
    df_counties['County Name'] = [(Get_CountyNames_Dict('51'))[x] for x in df_counties['County Code']]
    
    return df_counties

In [13]:
def Prelim_Clean_Vacc_By_Sex_Data(df_orig):
    df = df_orig.copy()
    df.set_index(['County Code', 'Report Date'], inplace = True)
    df.sort_index(inplace = True)
    
    col_list = ['At Least One Dose', 'Fully Vaccinated', 'First Booster', 'Second Booster', 
                'Immunocompromised Dose', 'Not Reported']
    for col in col_list:
        dcol = 'Diff ' + col
        dcol_sum = 'Sum Diff ' + col
        df[dcol] = df.groupby(level = 'County Code')[col].diff().fillna(0).astype(int)
        df[dcol_sum] = (df.groupby(level = 'County Code', as_index = False)[dcol].rolling(2).sum().
                        shift(periods = -1).fillna(0).astype(int).drop('County Code', axis = 1))
        mask = (df[dcol] < 0) & ((df[dcol_sum] >= 0) | (abs(df[dcol_sum]) < 0.05*abs(df[dcol])))
        Nentries = len(df[mask])
        if Nentries == 0:
            print('No entries to be adjusted for ', col)
            continue
        df.loc[mask, col] = np.nan
        df[col] = (df[col].ffill()).astype(int)
        print('{} entries adjusted for {}'.format(Nentries, col))
        df.drop(columns = [dcol, dcol_sum], inplace = True)
        
    df.reset_index(inplace = True)  
    return df

In [14]:
def Clean_Vacc_By_Sex_Data(df_orig):
    df = df_orig.copy()
    
    df.sort_values(by = ['County Code', 'Report Date'], inplace = True)
    date0 = df.iloc[0]['Report Date']
    df.set_index(['County Code', 'Report Date'], inplace = True)
    df.sort_index(inplace = True)
    
    col_list = df.columns.to_list()
    
    def Get_Obsolete_Date(df_col):
        df_col = df_col.to_frame().reset_index()
        df_col[col + ' bool'] = 1 - (df_col[col].astype(bool)).astype(int)
        df_bool = df_col.groupby('Report Date')[col + ' bool'].sum()
        return df_bool[df_bool == 0].index[-1]
    
    def Get_Dates_of_Adjusment(Mask):
        try:
            Adjusted_dates = (df[Mask].index).tolist()
        except KeyError:
            Adjusted_dates = []
        return Adjusted_dates
    
    for col in col_list:
        # First find any obsolete dates. Do not adjust after this date
        Obsolete_date = Get_Obsolete_Date(df[col].copy())
        
        new_col = 'New ' + col
        df[new_col] = df.groupby('County Code')[col].diff().fillna(0)

        print('Cleaning', col, 'data')
        Adjusted_dates_init = Get_Dates_of_Adjusment(df[new_col] < 0)
        Adjusted_dates = [date for date in Adjusted_dates_init if date[1] <= Obsolete_date]

        iter = 0
        for d in Adjusted_dates:
            Max_cases = df.loc[d, col]
            Mask = df[col] > Max_cases
            Ind = df[Mask].loc[(d[0], date0) : d].index
            df.loc[Ind, col] = Max_cases
            iter += 1

        print(iter, ' entries adjusted')
        df.drop(columns = [new_col], inplace = True)
#         df[new_col] = df.groupby('County Code')[col].diff().fillna(0).astype('int')

    df.reset_index(inplace = True)    
    return df

# **<font color = 'DarkRed'> Congressional Districts Level </font>**

In [15]:
def Get_CongressionalDistricts_COVID_data(State_Code, df_counties):
    CD_Pop = Get_District_County_Pop19(State_Code)
    col_list = df_counties.loc[:, 'New Daily Cases':'Deaths per Hospitalizations'].columns.to_list()
    df = pd.merge(CD_Pop.drop(columns = ['County Name']), 
                  df_counties.drop(columns = col_list), 
                  how = 'inner', on = 'County Code')

    df.set_index(['County Code', 'CDistrict'], inplace = True)
    
    Measures = ['Total Cases', 'Hospitalizations', 'Deaths']
    for col in Measures:
        m = col.replace('Total ', '')
        m = m.replace('Hospitalizations', 'Hosps')
        df.loc[:, 'Est ' + m + ' in District'] = df.loc[:, col].multiply(df['Ratio in District'], axis = 'index')
        df[col + ' in District'] = df['Est ' + m + ' in District'].round().astype(int)
        df['Diff ' + m] = (df.groupby(['County Code', 'Report Date'])[col + ' in District'].transform(sum) - df[col])
        df['Frac ' + m] = df['Est ' + m + ' in District']%1
        mask = df['Frac ' + m] <= 0.5
        df['Frac down ' + m] = df['Frac ' + m]*mask
        df['Frac up ' + m] = df['Frac ' + m] + mask
        df['isMax down ' + m] = ((df.groupby(['County Code', 'Report Date'])['Frac down ' + m].
                                 transform(max) == df['Frac down ' + m]) & (df['Frac down ' + m] != 0.0)).astype(int)
        df['isMin up ' + m] = ((df.groupby(['County Code', 'Report Date'])['Frac up ' + m].
                               transform(min) == df['Frac up ' + m]) & (df['Frac up ' + m] < 1.0)).astype(int)
        df[col + ' in District'] -= (df['isMax down ' + m] + df['isMin up ' + m])*df['Diff ' + m]
    
        df['Diff ' + m] = (df.groupby(['County Code', 'Report Date'])[col + ' in District'].transform(sum) - df[col])
        df['New ' + m + ' in District'] = (df.groupby(['County Code', 'CDistrict'])[col + ' in District']
                                          ).diff().fillna(0).astype(int)
    
        def Get_Locations_to_Adjust(Mask):
            try:
                adj_locs = (df[Mask].index).tolist()
            except KeyError:
                adj_locs = []
            return adj_locs
    
        df.reset_index(inplace = True)
        df.set_index(['County Code', 'CDistrict', 'Report Date'], inplace = True)
        adj_mask = df['New ' + m + ' in District'] < 0
        locs_to_adjust = Get_Locations_to_Adjust(adj_mask)

        for locs in locs_to_adjust:
            county = locs[0]
            district = locs[1]
            date = locs[2]
            dg = df.loc[(county, slice(None), date), :].sort_values(by = 'Ratio in District', ascending = False)
            all_districts = dg.index.get_level_values(1).tolist()
        
            df.loc[locs, col + ' in District'] -= df.loc[locs, 'New ' + m + ' in District']
        
            for d in all_districts:
                if df.loc[(county, d, date), 'New ' + m + ' in District'] >= - df.loc[locs, 'New ' + m + ' in District']:
                    df.loc[(county, d, date), col + ' in District'] += df.loc[locs, 'New ' + m + ' in District']
                    district2 = d
                    break
                
            mask = (df[col] == df.loc[locs, col]) & (df.index.get_level_values(0) == county)
            dates = df[mask].index.get_level_values(2).unique().tolist()
            dates.remove(date)
        
            for d in dates:
                df.loc[(county, district, d), col + ' in District'] = df.loc[(county, district, date), col + ' in District']
                df.loc[(county, district2, d), col + ' in District'] = df.loc[(county, district2, date), col + ' in District']
                        

        df['New ' + m + ' in District'] = (df.groupby(['County Code', 'CDistrict'])[col + ' in District']
                                          ).diff().fillna(0).astype(int)

    df_districts = df.groupby(['CDistrict', 'Report Date']
                             )[['Total Cases in District', 'Hospitalizations in District', 'Deaths in District']].sum()
    
    df_districts.columns = ['Total Cases', 'Hospitalizations', 'Deaths']
    df_districts['New Daily Cases'] = df_districts.groupby('CDistrict')['Total Cases'].diff().fillna(0).astype('int')
    
    df_districts['New Daily Hospitalizations'] = (df_districts.groupby('CDistrict')['Hospitalizations'].
                                                  diff().fillna(0).astype('int'))
    
    df_districts['New Daily Deaths'] = df_districts.groupby('CDistrict')['Deaths'].diff().fillna(0).astype('int')
    
    df_districts['Hospitalizations Ratio'] = (df_districts['Hospitalizations'].
                                              divide(df_districts['Total Cases'])).replace(np.nan, 0)
    df_districts['Deaths Ratio'] = (df_districts['Deaths'].divide(df_districts['Total Cases'])).replace(np.nan, 0)
    df_districts['Deaths per Hospitalizations'] = (df_districts['Deaths'].
                                                   divide(df_districts['Hospitalizations'])).replace(np.nan, 0)
    df_districts.reset_index(inplace = True)
    
    return df_districts 

In [16]:
def Get_CongressionalDistricts_COVID_Vaccination_data(State_Code, df_counties):
    CD_Pop = Get_District_County_Pop19(State_Code)
    col_list = df_counties.loc[:, 'people with at least 1 dose per Pop':].columns.to_list()
    df = pd.merge(CD_Pop.drop(columns = ['County Name']), 
                  df_counties.reset_index().drop(columns = col_list), 
                  how = 'inner', on = 'County Code')

    df.set_index(['County Code', 'CDistrict'], inplace = True)
    
    Measures = df.loc[:, 'people with at least 1 dose':].columns.to_list()
    for col in Measures:
        df.loc[:, 'Est ' + col + ' in District'] = df.loc[:, col].multiply(df['Ratio in District'], axis = 'index')
        df[col + ' in District'] = df['Est ' + col + ' in District'].round().astype(int)
        df['Diff ' + col] = (df.groupby(['County Code', 'Date'])[col + ' in District'].transform(sum) - df[col])
        df['Frac ' + col] = df['Est ' + col + ' in District']%1
        mask = df['Frac ' + col] <= 0.5
        df['Frac down ' + col] = df['Frac ' + col]*mask
        df['Frac up ' + col] = df['Frac ' + col] + mask
        df['isMax down ' + col] = ((df.groupby(['County Code', 'Date'])['Frac down ' + col].
                                    transform(max) == df['Frac down ' + col]) & 
                                   (df['Frac down ' + col] != 0.0)).astype(int)
        df['isMin up ' + col] = ((df.groupby(['County Code', 'Date'])['Frac up ' + col].
                                  transform(min) == df['Frac up ' + col]) & 
                                 (df['Frac up ' + col] < 1.0)).astype(int)
        df[col + ' in District'] -= (df['isMax down ' + col] + df['isMin up ' + col])*df['Diff ' + col]
    
        df['Diff ' + col] = (df.groupby(['County Code', 'Date'])[col + ' in District'].transform(sum) - df[col])
        df['New ' + col + ' in District'] = (df.groupby(['County Code', 'CDistrict'])
                                             [col + ' in District']).diff().fillna(0).astype(int)
    
        def Get_Locations_to_Adjust(Mask):
            try:
                adj_locs = (df[Mask].index).tolist()
            except KeyError:
                adj_locs = []
            return adj_locs
    
        df.reset_index(inplace = True)
        df.set_index(['County Code', 'CDistrict', 'Date'], inplace = True)
        adj_mask = df['New ' + col + ' in District'] < 0
        locs_to_adjust = Get_Locations_to_Adjust(adj_mask)

        for locs in locs_to_adjust:
            county = locs[0]
            district = locs[1]
            date = locs[2]
            dg = df.loc[(county, slice(None), date), :].sort_values(by = 'Ratio in District', ascending = False)
            all_districts = dg.index.get_level_values(1).tolist()
        
            df.loc[locs, col + ' in District'] -= df.loc[locs, 'New ' + col + ' in District']
        
            for d in all_districts:
                if df.loc[(county, d, date), 'New ' + col + ' in District'] >= -df.loc[locs, 'New ' + col + ' in District']:
                    df.loc[(county, d, date), col + ' in District'] += df.loc[locs, 'New ' + col + ' in District']
                    district2 = d
                    break
                
            mask = (df[col] == df.loc[locs, col]) & (df.index.get_level_values(0) == county)
            dates = df[mask].index.get_level_values(2).unique().tolist()
            dates.remove(date)
        
            for d in dates:
                df.loc[(county, district, d), col + ' in District']=df.loc[(county, district, date), col + ' in District']
                df.loc[(county, district2, d), col + ' in District']=df.loc[(county, district2, date), col + ' in District']
                        

        df['New ' + col + ' in District'] = (df.groupby(['County Code', 'CDistrict'])[col + ' in District']
                                            ).diff().fillna(0).astype(int)

    cols = [s + ' in District' for s in Measures]
    df_districts = df.groupby(['CDistrict', 'Date'])[cols].sum()
    df_districts.columns = Measures
    df_districts.reset_index(inplace = True)
    df_districts = Add_Pop_data(State_Code, 'Congressional District', df_districts, Measures)
    df_districts.set_index('Date', inplace = True)
    
    return df_districts 

In [17]:
def Get_CongressionalDistricts_COVID_Vacc_Stat_data(State_Code, df_counties):
    CD_Pop = Get_District_County_Pop19(State_Code)
    col_list = [col for col in df_counties.columns if ('per Pop' in col) | (col == 'County Name')]
    df = pd.merge(CD_Pop.drop(columns = ['County Name']), 
                  df_counties.reset_index().drop(columns = col_list), 
                  how = 'inner', on = 'County Code')

    df.set_index(['County Code', 'CDistrict'], inplace = True)
    
    Measures = (df_counties.reset_index().drop(columns = col_list).
                set_index(['County Code', 'Date']).columns.to_list())
    
    def Get_Obsolete_Date(df_col):
        df_col = df_col.to_frame().reset_index()
        df_col[col + ' bool'] = 1 - (df_col[col].astype(bool)).astype(int)
        df_bool = df_col.groupby('Date')[col + ' bool'].sum()
        return df_bool[df_bool == 0].index[-1]

    for col in Measures:
        df.loc[:, 'Est ' + col + ' in District'] = df.loc[:, col].multiply(df['Ratio in District'], axis = 'index')
        df[col + ' in District'] = df['Est ' + col + ' in District'].round().astype(int)
        df['Diff ' + col] = (df.groupby(['County Code', 'Date'])[col + ' in District'].transform(sum) - df[col])
        df['Frac ' + col] = df['Est ' + col + ' in District']%1
        mask = df['Frac ' + col] <= 0.5
        df['Frac down ' + col] = df['Frac ' + col]*mask
        df['Frac up ' + col] = df['Frac ' + col] + mask
        df['isMax down ' + col] = ((df.groupby(['County Code', 'Date'])['Frac down ' + col].
                                    transform(max) == df['Frac down ' + col]) & 
                                   (df['Frac down ' + col] != 0.0)).astype(int)
        df['isMin up ' + col] = ((df.groupby(['County Code', 'Date'])['Frac up ' + col].
                                  transform(min) == df['Frac up ' + col]) & 
                                 (df['Frac up ' + col] < 1.0)).astype(int)
        df[col + ' in District'] -= (df['isMax down ' + col] + df['isMin up ' + col])*df['Diff ' + col]
    
        df['Diff ' + col] = (df.groupby(['County Code', 'Date'])[col + ' in District'].transform(sum) - df[col])
        df['New ' + col + ' in District'] = (df.groupby(['County Code', 'CDistrict'])
                                             [col + ' in District']).diff().fillna(0).astype(int)
        
        def Get_Locations_to_Adjust(Mask):
            try:
                adj_locs = (df[Mask].index).tolist()
            except KeyError:
                adj_locs = []
            return adj_locs
    
        df.reset_index(inplace = True)
        df.set_index(['County Code', 'CDistrict', 'Date'], inplace = True)
        
        # First find any obsolete dates. Do not adjust after this date
        Obsolete_date = Get_Obsolete_Date(df[col].copy())        
        
        adj_mask = (df['New ' + col + ' in District'] < 0) & (df.index.get_level_values(2) <= Obsolete_date)
        locs_to_adjust = Get_Locations_to_Adjust(adj_mask)

        for locs in locs_to_adjust:
            county = locs[0]
            district = locs[1]
            date = locs[2]
            dg = df.loc[(county, slice(None), date), :].sort_values(by = 'Ratio in District', ascending = False)
            all_districts = dg.index.get_level_values(1).tolist()
        
            df.loc[locs, col + ' in District'] -= df.loc[locs, 'New ' + col + ' in District']
        
            for d in all_districts:
                if df.loc[(county, d, date), 'New ' + col + ' in District'] >= -df.loc[locs, 'New ' + col + ' in District']:
                    df.loc[(county, d, date), col + ' in District'] += df.loc[locs, 'New ' + col + ' in District']
                    district2 = d
                    break
                
            mask = (df[col] == df.loc[locs, col]) & (df.index.get_level_values(0) == county)
            dates = df[mask].index.get_level_values(2).unique().tolist()
            dates.remove(date)
        
            for d in dates:
                df.loc[(county, district, d), col + ' in District']=df.loc[(county, district, date), col + ' in District']
                df.loc[(county, district2, d), col + ' in District']=df.loc[(county, district2, date), col + ' in District']
                        

        df['New ' + col + ' in District'] = (df.groupby(['County Code', 'CDistrict'])[col + ' in District']
                                            ).diff().fillna(0).astype(int)

    cols = [s + ' in District' for s in Measures]
    df_districts = df.groupby(['CDistrict', 'Date'])[cols].sum()
    df_districts.columns = Measures
    df_districts.reset_index(inplace = True)
    df_districts = Add_Pop_data(State_Code, 'Congressional District', df_districts, Measures)
    df_districts.set_index('Date', inplace = True)
    
    return df_districts 

# <font color='red'>=============================================================</font>

In [18]:
def Get_Per_Pop(State_Code, scope, df, col):
    if scope == 'County':
        Pop_df = Get_Counties_Pop19(State_Code)
    elif scope == 'Congressional District':
        Pop_df = Get_Districts_Pop19(State_Code)

    return df.loc[:, col]/Pop_df.loc[:, 'Population']

In [19]:
def Add_Pop_data(State_Code, level, df, col_list):
#     print(col_list)
    if level == 'County':
        Pop_df = Get_Counties_Pop19(State_Code)
        df.set_index('County Code', inplace = True)
    elif level == 'Congressional District':
        Pop_df = Get_Districts_Pop19(State_Code)
        df.set_index('CDistrict', inplace = True)
    
    df.sort_index(inplace = True)
    for col1 in col_list:
        col2 = col1 + ' per Pop'
        df[col2] = df.loc[:, col1]/Pop_df.loc[:, 'Population']
        
    df.reset_index(inplace = True)
    return df

In [20]:
def Get_Monthly_Data(df, sIndex):
    df_m = df.copy()

    df_m['Report Date'] = pd.to_datetime(df_m['Report Date'], yearfirst = True)
    df_m['Year'] = df_m['Report Date'].dt.year
    df_m['Month'] = df_m['Report Date'].dt.month
    
    cols = ['Monthly Cases', 'Monthly Hospitalizations', 'Monthly Deaths']
    dcols = {'New Daily Cases': cols[0], 'New Daily Hospitalizations': cols[1], 'New Daily Deaths': cols[2]}
    
    df_m = df_m.rename(columns = dcols).groupby(['Year', 'Month', sIndex])[cols].sum().reset_index()
    
    d = dict(enumerate(calendar.month_abbr))
    df_m.Month = df_m.Month.map(d)
    df_m.Year = df_m.Year.astype(str)
    df_m['Date'] = df_m.Year + '-' + df_m.Month
    
    df_m.set_index(sIndex, inplace = True)
    df_m.sort_index(inplace = True)
 
    return df_m

In [21]:
def Get_Weekly_Vacc_Data(df, group, freq):
    day_name_dict = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
    day_of_the_week = df.index[-1].dayofweek
    day = day_name_dict[day_of_the_week]
    df_weekly = df.groupby(group).resample(freq + day, closed = 'left').asfreq().reset_index(level = 0, drop = True)
    df_weekly.index.names = ['Date']
    
    return df_weekly.dropna()